# Ansible Playbooks
---

Playbooks can be used to manage configurations of and deployments to remote machines. 
- They can describe a policy you want your remote systems to enforce, or a set of steps in a general IT process. 
- Playbooks are designed to be human-readable and are developed in YAML. 
- There are multiple ways to organize playbooks and the files they include.

![Playbook](./notebook-files/playbook.png)


### Example playbook

Below you can see a fairly comrehensive playbook showing a wide-variety of what is possible in Ansible. This includes:

- registering the output of a task
- using a conditional
- running a task as the root user
- preventing output to the console
- using built-in Ansible variables
- using the vars feature of a playbook
- using Jinja2 filters
- etc.

Be sure to take note of this playbook and its output as we will be seeing it later.

In [ ]:
wpb "ansible_playbook_demo.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    sosreport_debug: True
    my_email_address: "nobody@localhost"

  tasks:
  - name: install sosreport package
    package:
      name: sosreport
      state: latest
    become: true

  - name: run sosreport
    command: "sos report -l"
    register: sosreport_output
    become: true
    no_log: true

  - name: print the output
    debug:
      msg: "{{ sosreport_output.stdout_lines }}"
    when: sosreport_debug

  - name: email me the results
    mail:
      host: localhost
      port: 25
      to: "{{ my_email_address }}"
      subject: "SOS Report for {{ ansible_default_ipv4.address }} on {{ ansible_date_time.date }}"
      body: "{{ sosreport_output.stdout_lines | to_nice_json }}"
...
############################################################

ansible-playbook "ansible_playbook_demo.yml"

## Utilities
---

### lineinfile

Next we will look at a playbook to insert or replace text in a file using the Ansible Module `lineinfile`

> Commonly used to edit text file and configuration files, for example:
> - a simple httpd.conf configuration for a Web Server
> - an authorized keys files to set passwordless ssh log into a Linux server
> - a revisit of the hosts file example.


#### Example 1 - a configuration file

> - Run the cell to create the file
> - Run the playbook and see how the values are targetted and changed

In [ ]:
cat << EOF > "/home/cvx_admin_user/httpd.conf"

Listen 80
ServerAdmin admin@localhost

EOF

In [ ]:
wpb "ansible_playbook_lineinfile_httpd.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    my_admin_account_email: "my_admin_account@chevron.com"

  tasks:
    - name: Change listen on port 80 to listen on port 443
      lineinfile:
        path: "/home/cvx_admin_user/httpd.conf"
        regexp: "^Listen"
        line: "Listen 443"

    - name: Change admin email
      lineinfile:
        path: "/home/cvx_admin_user/httpd.conf"
        regexp: "^ServerAdmin"
        line: "ServerAdmin {{ my_admin_account_email }}"

...
############################################################

# Show the file before
cat "/home/cvx_admin_user/httpd.conf"

ansible-playbook "ansible_playbook_lineinfile_httpd.yml"

# Show the file after
cat "/home/cvx_admin_user/httpd.conf"

#### Example 2 - ssh authorized keys file


> - Run the cell to create the file
> - Run the playbook and see how the vaules are targetted and changed
> - Note the loop to edit the same file multiple times ensure list values are added


In [ ]:
cat << EOF > "/home/cvx_admin_user/authorized_keys"

ssh-rsa AAA... key1@chevron.com
ssh-rsa AAB... key2@chevron.com
EOF

In [ ]:
cd ~/Ansible-Tutorial
wpb "ansible_playbook_lineinfile_auth.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    authorized_keys_file: "/home/cvx_admin_user/authorized_keys"
    my_ssh_keys:
    - "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABA... phcr@chevron.com"
    - "ssh-rsa AAAAB3NzaC1yc2EAABBBCCCDD... mcpu@chevron.com"
    - "ssh-rsa AABBBDDDEC1yc2EAABBBCCCDD... pdeg@chevron.com"
    - "ssh-rsa AABBBDDDEC1yc2EAAFFFCCCDD... ngfx@chevron.com"

  tasks:
    - name: add my ssh keys to the VM
      lineinfile:
        path: "{{ authorized_keys_file }}"
        insertafter: EOF
        line: "{{ item }}"
      loop: "{{ my_ssh_keys }}"

...
############################################################

# Show the file before
cat "/home/cvx_admin_user/authorized_keys"

ansible-playbook "ansible_playbook_lineinfile_auth.yml"

# Show the file after
cat "/home/cvx_admin_user/authorized_keys"

#### Example 3 - a hosts file revisited 


> - Run the cell to create the file
> - Run the playbook and see how the line we want is added to the file

In [ ]:
cat << EOF > "/home/cvx_admin_user/hosts2"
# Hosts File
EOF

In [ ]:
wpb "ansible_playbook_lineinfile_hosts.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
  - name: add my machine information to hosts file
    lineinfile:
      path: "/home/cvx_admin_user/hosts2"
      insertafter: EOF
      line: "{{ ansible_default_ipv4.address }} {{ ansible_facts.fqdn }}"
...
############################################################

# Show the file before
cat "/home/cvx_admin_user/hosts2"

ansible-playbook "ansible_playbook_lineinfile_hosts.yml"

# Show the file after
cat "/home/cvx_admin_user/hosts2"

### Templating

Last we will look at a Jinja2 template as is commonly used alongside Ansible.

> Note:
> - you can template anywhere, even inside comments!

Look at motd - message of the day example.  A common warning template but includes custom variables for phone and email. 

In [ ]:
cat << EOF > "/home/cvx_admin_user/notebooks/playbooks/templates/motd.j2"


**************************************************************************

NOTICE TO USERS: This {{ company_name }} system is for use by authorized users only.
To the extent permitted by law, by using this system you acknowledge and
consent to the monitoring, access, use, and disclosure by Chevron of any
information generated, received, or stored on the system and waive any right
or expectation of privacy in connection with your use. Unauthorized and/or
improper use of this system in violation of Chevron corporate policies is
not tolerated and may result in disciplinary action, including termination.


Current Hostname: {{ ansible_facts.fqdn }}  Current IP Address: {{  ansible_default_ipv4.address }}
For Assistance email: {{ help_desk_email }} or call {{ help_desk_phone }}

**************************************************************************

EOF


In [ ]:
## NOTE THESE ARE BOGUS EMAIL ADDRESS FOR DEMONSTRATION ONLY

wpb "ansible_template_demo1.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    company_name: "Chevron"
    help_desk_email: "local-helpdesk-usa@chevron.com"
    help_desk_phone: "CTN: 8-8765"

# Note using the template module it will look under templates directory by default

  tasks:

  - name: Process the template and copy it into place
    template:
      src: "motd.j2"
      dest: "/etc/motd"
      owner: "root"
      group: "root"
      mode: 0644
    become: true
...
############################################################

cat "/home/cvx_admin_user/notebooks/playbooks/templates/motd.j2"

ansible-playbook "ansible_template_demo1.yml"

cat "/etc/motd"

#### Templating - Example 2

Using the previous example for httpd.conf seen in the lineinfile section

In [ ]:
cat << EOF > "/home/cvx_admin_user/notebooks/playbooks/templates/httpd.conf.j2"

Listen {{ listen_port }}
ServerAdmin {{ my_admin_account_email }}

EOF

In [ ]:
wpb "ansible_template_demo2.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    my_admin_account_email: "my_admin_account@chevron.com"
    listen_port: 443

# Note using the template module it will look under templates directory by default

  tasks:
  - name: Process the template and copy it into place
    template:
      src: "httpd.conf.j2"
      dest: "/home/cvx_admin_user/httpd.conf2"
    become: true
...
############################################################

cat "/home/cvx_admin_user/notebooks/playbooks/templates/httpd.conf.j2"

ansible-playbook "ansible_template_demo2.yml"

cat "/home/cvx_admin_user/httpd.conf2"

## Further Explanation
---

**See Also:**
- [Playbooks](https://docs.ansible.com/ansible/latest/user_guide/playbooks.html)
- [Ansible Examples](https://github.com/ansible/ansible-examples)